In [1]:
pip install pandas requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd

In [21]:
API_KEY = '54d77ee15672ad11f701914343001ba1'

In [22]:
def fetch_fred_data(series_id, start_date='1978-01-01', end_date='2023-12-31'):
    url = "https://api.stlouisfed.org/fred/series/observations"
    params = {
        'series_id': series_id,
        'api_key': API_KEY,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date,
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"Failed to fetch data for {series_id}. HTTP Status Code: {response.status_code}")
        return pd.DataFrame()

    data = response.json()
    observations = pd.DataFrame(data.get('observations', []))

    if not observations.empty:
        observations['date'] = pd.to_datetime(observations['date'])
        observations['value'] = pd.to_numeric(observations['value'], errors='coerce')
        return observations[['date', 'value']].rename(columns={'value': series_id})
    else:
        print(f"No data available for {series_id}.")
        return pd.DataFrame()


In [ ]:
fred_series = {
    'GDP': 'gdp_growth_rate',                  
    'UNRATE': 'unemployment_rate',             
    'CPIAUCSL': 'inflation_cpi',               
    'INDPRO': 'industrial_production',         
    'T10Y2Y': 'yield_curve_spread',            
    # 'SP500': 'sp500_index', data only available from 2015 - least significant indicator                   
    # 'BAA10Y': 'credit_spread',                 
    'UMCSENT': 'consumer_confidence',          
}

In [24]:
def align_frequencies(data, target_frequency='M', method='mean'):
    if method == 'mean':
        return data.resample(target_frequency).mean()
    elif method == 'sum':
        return data.resample(target_frequency).sum()
    elif method == 'last':
        return data.resample(target_frequency).last()
    else:
        raise ValueError(f"Unsupported method: {method}")


In [25]:
def fetch_and_compile_data(series_dict, start_date='2000-01-01', end_date='2023-12-31', target_frequency='M'):
    all_data = []

    for series_id, column_name in series_dict.items():
        print(f"Fetching and processing data for {column_name} ({series_id})...")
        data = fetch_fred_data(series_id, start_date, end_date)
        if not data.empty:
            data = data.rename(columns={series_id: column_name})
            data = data.set_index('date')
            if target_frequency == 'M':
                data = align_frequencies(data, target_frequency='M', method='mean')
            elif target_frequency == 'Q':
                data = align_frequencies(data, target_frequency='Q', method='mean')
            all_data.append(data)

    combined_data = pd.DataFrame()
    if all_data:
        combined_data = all_data[0]
        for df in all_data[1:]:
            combined_data = pd.merge(combined_data, df, left_index=True, right_index=True, how='outer')

    combined_data.sort_index(inplace=True)
    combined_data.fillna(method='ffill', inplace=True)
    combined_data.fillna(method='bfill', inplace=True)
    return combined_data


In [26]:
labels = list(fred_series.keys())

In [ ]:
combined_dataset = fetch_and_compile_data(fred_series, start_date='1978-01-01', end_date='2024-11-01', target_frequency='M')
combined_dataset.to_csv('standardized_economic_indicators.csv')
print("Data saved to 'standardized_economic_indicators.csv'")

Fetching and processing data for gdp_growth_rate (GDP)...
Fetching and processing data for unemployment_rate (UNRATE)...
Fetching and processing data for inflation_cpi (CPIAUCSL)...
Fetching and processing data for industrial_production (INDPRO)...
Fetching and processing data for yield_curve_spread (T10Y2Y)...
Fetching and processing data for credit_spread (BAA10Y)...
Fetching and processing data for consumer_confidence (UMCSENT)...
Data saved to 'standardized_economic_indicators_cs.csv'


C:\Users\Tomás\AppData\Local\Temp\ipykernel_20916\2263320569.py:23: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_data.fillna(method='ffill', inplace=True)
C:\Users\Tomás\AppData\Local\Temp\ipykernel_20916\2263320569.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_data.fillna(method='bfill', inplace=True)


In [28]:
combined_dataset.columns=labels
combined_dataset

,GDP,UNRATE,CPIAUCSL,INDPRO,T10Y2Y,BAA10Y,UMCSENT
date,,,,,,,
1986-01-31,4507.894,6.7,109.900,55.6453,1.051429,2.241429,95.6
1986-02-28,4507.894,7.2,109.700,55.2634,0.736842,2.408421,95.9
1986-03-31,4507.894,7.2,109.100,54.8728,0.564000,2.716500,95.1
1986-04-30,4545.340,7.1,108.700,54.9350,0.604091,2.891364,96.2
1986-05-31,4545.340,7.2,109.000,55.0399,0.642381,2.576190,94.8
...,...,...,...,...,...,...,...
2024-07-31,29349.924,4.3,313.534,102.5381,-0.247273,1.588182,66.4
2024-08-31,29349.924,4.2,314.121,103.0449,-0.094545,1.733636,67.9
2024-09-30,29349.924,4.1,314.686,102.5497,0.101000,1.694500,70.1


In [ ]:
combined_dataset.to_csv('KEI_recession.csv')
print("Data saved to 'KEI_recession.csv'")

Data saved to 'KEI_recession_cs.csv'
